In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

print(tf.__version__)

2.1.0


In [2]:
from tensorflow.keras import datasets, layers, models

In [3]:
from tensorflow.python.keras.layers import Conv2D

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 

In [5]:
import numpy as np

In [6]:
import os,shutil,glob
from tqdm.notebook import tqdm

In [7]:
from skimage.transform import resize

In [12]:
colored_img_files = glob.glob("./data/coloredimg/*.jpg")
non_colored_img_files = glob.glob("./data/bwimg/*.jpg")

In [13]:
print(colored_img_files[0:10])
print(non_colored_img_files[0:10])

['./data/coloredimg/airplane_0412.jpg', './data/coloredimg/flower_0045.jpg', './data/coloredimg/flower_0723.jpg', './data/coloredimg/airplane_0374.jpg', './data/coloredimg/airplane_0360.jpg', './data/coloredimg/flower_0737.jpg', './data/coloredimg/fruit_0819.jpg', './data/coloredimg/person_0478.jpg', './data/coloredimg/flower_0051.jpg', './data/coloredimg/airplane_0406.jpg']
['./data/bwimg/airplane_0412.jpg', './data/bwimg/flower_0045.jpg', './data/bwimg/flower_0723.jpg', './data/bwimg/airplane_0374.jpg', './data/bwimg/airplane_0360.jpg', './data/bwimg/flower_0737.jpg', './data/bwimg/fruit_0819.jpg', './data/bwimg/person_0478.jpg', './data/bwimg/flower_0051.jpg', './data/bwimg/airplane_0406.jpg']


In [95]:
X = []
Y = []
for cimgfile in tqdm(colored_img_files[0:20]):
    bwimgfile = "./data/bwimg/" + cimgfile[len("./data/coloredimg/"):]
    cimage = mpimg.imread(cimgfile)
    bwimage = mpimg.imread(bwimgfile)
    
    cimage_resized = resize(cimage, (150, 150,3),
                       anti_aliasing=True)
    bwimage_resized = resize(bwimage, (150, 150),
                       anti_aliasing=True)
#     Y.append(np.array(cimage_resized).flatten())
    Y.append(bwimage_resized)
    X.append(list(np.resize(bwimage_resized,(150,150,1))))
    

In [96]:
np.shape(Y)

(20, 150, 150)

In [59]:
from sklearn.model_selection import train_test_split

In [97]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.2,
    shuffle=True,
    random_state=42,
)
# X_train, X_val, y_train, y_val = train_test_split(
#     X_train, 
#     y_train, 
#     test_size=0.2, 
#     random_state=42)

X_train = np.array(X_train)
X_test = np.array(X_test)
# X_val = np.array(X_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
# y_val = np.array(y_val)

In [85]:
type(X_train)

numpy.ndarray

In [98]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150,1),padding = "SAME"))
# model.add(layers.MaxPooling2D((1,1)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150,1),padding = "SAME"))
model.add(layers.Conv2D(1, (3, 3), activation='relu', input_shape=(150, 150,1),padding = "SAME"))
model.add(layers.MaxPooling2D((1,1)))

In [99]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 150, 150, 32)      320       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 150, 150, 64)      18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 150, 150, 1)       577       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 150, 150, 1)       0         
Total params: 19,393
Trainable params: 19,393
Non-trainable params: 0
_________________________________________________________________


In [100]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [101]:
history = model.fit(X_train, y_train, epochs=10)

Train on 16 samples
Epoch 1/10
16/16 [==============================] - 10s 653ms/sample


InvalidArgumentError:  logits and labels must have the same first dimension, got logits shape [2400,150] and labels shape [360000]
	 [[node loss/max_pooling2d_2_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-101-8e290d6dcd1f>:1) ]] [Op:__inference_distributed_function_2088]

Function call stack:
distributed_function
